In [1]:
import pandas as pd
import json

In [20]:
import json

with open("/home/amedeo/Desktop/thesis_project/thesis/data/good_tokens/token_analysis_0x000000_full_20250820_202916.json") as f:
    gooddata = json.load(f)
with open ("/home/amedeo/Desktop/thesis_project/thesis/data/bad_tokens/token_analysis_0x0AdA00_full_20250804_122718.json") as f:
    baddata = json.load(f)

# Contract

In [24]:
data = baddata
# Extract contract data
contract_data = {
    'token_address': data['token_address'],
    'chain': data['chain'],
    'token_name': data['token_name'],
    **data['analyses']['contract']
}

# Create contract dataframe
contract_df = pd.DataFrame([contract_data])

In [25]:
contract_df.dtypes

token_address         object
chain                 object
token_name            object
contract_name         object
compiler_version      object
license_type          object
implementation        object
source_code           object
abi                   object
verified                bool
owner                 object
creator               object
is_hidden_owner         bool
is_proxy                bool
is_sellable             bool
is_hardcoded_owner      bool
code_analysis         object
dtype: object

# Holders

In [28]:
data = baddata
# Extract holders data (excluding holders_list)
holders_data = {
    'token_address': data['token_address'],
    'chain': data['chain'],
    'token_name': data['token_name'],
    **{k: v for k, v in data['analyses']['holder'].items() if k != 'holders_list'}
}

# Create holders dataframe
holders_df = pd.DataFrame([holders_data])

In [29]:
holders_df.dtypes

token_address                                  object
chain                                          object
token_name                                     object
total_holders                                   int64
total_supply                                  float64
total_circulating_supply                      float64
owner                                          object
creator                                        object
holders_exceeding_5_percent                    object
howmany_holders_exceeding_5_percent             int64
top_10_holders                                 object
total_top_10_balance                          float64
top10_percentage_of_total_supply              float64
top10_percentage_of_circulating_supply        float64
top_10_less_than_70_percent_of_total             bool
top_10_less_than_70_percent_of_circulating       bool
dtype: object

## Holders list for an address

In [38]:
data = gooddata
# Extract holders_list data
holders_list_data = []
if 'holders_list' in data['analyses']['holder'] and data['analyses']['holder']['holders_list']:
    for address, holder_info in data['analyses']['holder']['holders_list'].items():
        row = {
            'token_address': data['token_address'],
            'chain': data['chain'],
            'token_name': data['token_name'],
            'holder_address': address,
            **holder_info
        }
        holders_list_data.append(row)
else:
    # Add empty row if no holders_list
    row = {
        'token_address': data['token_address'],
        'chain': data['chain'],
        'token_name': data['token_name'],
        'holder_address': None,
        'balance': None,
        'percentage_of_total_supply': None,
        'percentage_of_circulating_supply': None
    }
    holders_list_data.append(row)

# Create holders_list dataframe
holders_list_df = pd.DataFrame(holders_list_data)

In [39]:
holders_list_df['balance']

0     431232921022684739928064
1       4630581226041445974016
2        216665075325600006144
3        174931577905897144320
4         48874999993083142144
5         23782996130323439616
6         22499999997320499200
7          9430101406738311168
8          6011099373153843200
9          5391478333090949120
10         4321918767881402368
11         2637051639490242048
12         2033333984766862336
13         1635570334891500032
14         1299170131625090048
15          154828384035756960
16          151984617139317728
17            9810077652768224
18                           5
19                           2
20                           1
21                           1
22                           1
Name: balance, dtype: object

In [8]:
grouped_list = holders_list_df.groupby('token_address').agg({'balance': 'mean', 'holder_address': 'count'}).reset_index()

In [9]:
holders_df.merge(grouped_list, on="token_address", how="left")

,token_address,chain,token_name,total_holders,total_supply,total_circulating_supply,owner,creator,holders_exceeding_5_percent,howmany_holders_exceeding_5_percent,top_10_holders,total_top_10_balance,top10_percentage_of_total_supply,top10_percentage_of_circulating_supply,top_10_less_than_70_percent_of_total,top_10_less_than_70_percent_of_circulating,balance,holder_address
0,0xE60cab134DFB46FF7E3588E9664996fc7ae946bA,bsc,Kelon MBY,80,100000000000000000000,100000000000000000000,{'address': '0x386b5c1692b640d37a556186ddeffff...,{},[{'address': '0x386b5c1692b640d37a556186ddefff...,1,[{'address': '0x386b5c1692b640d37a556186ddefff...,99999992523135247903,99.999993,99.999993,False,False,1.223387e+13,79


# Liquidity

In [10]:
# Extract liquidity data (excluding lp_holders)
liquidity_data = {
    'token_address': data['token_address'],
    'chain': data['chain'],
    'token_name': data['token_name'],
    **{k: v for k, v in data['analyses']['liquidity'].items() if k != 'lp_holders'}
}

# Create liquidity dataframe
liquidity_df = pd.DataFrame([liquidity_data])

In [24]:
liquidity_df.head()

,token_address,chain,token_name,price_usd,liquidity_usd,market_cap_usd,liquidity_to_market_cap_ratio,token_volume,volume_usd,volume_to_liquidity_ratio,locked_liquidity_percent,locked_95_for_15_days,creator_under_5_percent,creator_percent_of_lp,owner_under_5_percent,owner_percent_of_lp,total_lp_supply,lp_holders_count,error
0,0xE60cab134DFB46FF7E3588E9664996fc7ae946bA,bsc,Kelon MBY,5039.2191,2.756500e-08,5.039219e+23,5.470094e-32,0.0,0.0,0.0,0.0,False,False,99.999,True,0.0,5.276399e-09,2,Liquidity pool info could not be retrieved.


## Liquidity Pool Holders Dataframe

In [25]:
# Extract lp_holders data
lp_holders_data = []
if 'lp_holders' in data['analyses']['liquidity'] and data['analyses']['liquidity']['lp_holders']:
    for lp_holder in data['analyses']['liquidity']['lp_holders']:
        row = {
            'token_address': data['token_address'],
            'chain': data['chain'],
            'token_name': data['token_name'],
            **lp_holder
        }
        lp_holders_data.append(row)
else:
    # Add empty row if no lp_holders
    row = {
        'token_address': data['token_address'],
        'chain': data['chain'],
        'token_name': data['token_name'],
        'address': None,
        'balance': None,
        'is_locked': None,
        'percent': None,
        'tag': None
    }
    lp_holders_data.append(row)

# Create lp_holders dataframe
lp_holders_df = pd.DataFrame(lp_holders_data)

In [26]:
lp_holders_df.head()

,token_address,chain,token_name,address,balance,is_locked,percent,tag
0,0xE60cab134DFB46FF7E3588E9664996fc7ae946bA,bsc,Kelon MBY,0x0ed943ce24baebf257488771759f9bf482c39706,5.276398e-09,False,9.999998e-01,
1,0xE60cab134DFB46FF7E3588E9664996fc7ae946bA,bsc,Kelon MBY,0x0000000000000000000000000000000000000000,1.000000e-15,True,1.895232e-07,Null Address


# Security

In [27]:
# Extract security data
security_data = {
    'token_address': data['token_address'],
    'chain': data['chain'],
    'token_name': data['token_name'],
    **data['analyses']['security']
}

# Create security dataframe
security_df = pd.DataFrame([security_data])

In [28]:
security_df.head()

,token_address,chain,token_name,warnings,homany_warnings,suspicious_urls,howmany_suspicious_urls,suspicious_addresses,howmany_suspicious_addresses,error
0,0xE60cab134DFB46FF7E3588E9664996fc7ae946bA,bsc,Kelon MBY,None,0,None,0,None,0,Security analysis data not available.


# Lifecycle

In [29]:
# Extract lifecycle data
lifecycle_data = {
    'token_address': data['token_address'],
    'chain': data['chain'],
    'token_name': data['token_name'],
    **data['analyses']['lifecycle']
}

# Create lifecycle dataframe
lifecycle_df = pd.DataFrame([lifecycle_data])

In [30]:
lifecycle_df.head()

,token_address,chain,token_name,token_age_seconds,token_creation_date,creation_to_first_trade_seconds,creation_to_first_trade_blocks,last_tx_hash,last_active_age,inactive_days
0,0xE60cab134DFB46FF7E3588E9664996fc7ae946bA,bsc,Kelon MBY,6.822589e+07,2023-06-05T15:21:36,0.0,0,0x8888b37e97e41970e2bd9bcdab97b708ddfaab79a99b...,2024-04-17T06:35:12,473.017101
